## Toxic: Word2Vec

In [1]:
import time
import pandas as pd

import numpy as np
import tensorflow as tf
import pandas as pd

import utils

checkpoint = "checkpoints/w2v_3allwords.ckpt"

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_pickle('../data/toxictrain.pkl')
# df = df[df['rating']>0]
documents = [''.join(r) for r in df.comment_text]
text = ''.join(documents)

In [3]:
import re

text = re.sub('[^A-Za-z0-9 ]+', '',text)

In [4]:
words = utils.preprocess(text)
print(words[:30])

['i', 'hope', 'your', 'retarded', 'kids', 'get', 'anal', 'raped', 'and', 'murdered', 'for', 'having', 'such', 'a', 'fag', 'as', 'a', 'father', 'im', 'gonna', 'fuck', 'your', 'fat', 'wife', 'and', 'her', 'over', 'the', 'bridge', 'consider']


In [5]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 9910700
Unique words: 39064


In [6]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [7]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

In [8]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [9]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [10]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [11]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [12]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [13]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
# If the checkpoints directory doesn't exist:
# !mkdir checkpoints

In [ ]:
epochs = 50
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "../data/checkpoints/w2v_3allwords.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/50 Iteration: 100 Avg. Training loss: 5.7170 0.4361 sec/batch
Epoch 1/50 Iteration: 200 Avg. Training loss: 5.6756 0.4354 sec/batch
Epoch 1/50 Iteration: 300 Avg. Training loss: 5.6824 0.4011 sec/batch
Epoch 1/50 Iteration: 400 Avg. Training loss: 5.6789 0.4592 sec/batch
Epoch 1/50 Iteration: 500 Avg. Training loss: 5.6125 0.3727 sec/batch
Epoch 1/50 Iteration: 600 Avg. Training loss: 5.6280 0.3723 sec/batch
Epoch 1/50 Iteration: 700 Avg. Training loss: 5.5377 0.3715 sec/batch
Epoch 1/50 Iteration: 800 Avg. Training loss: 5.4435 0.3735 sec/batch
Epoch 1/50 Iteration: 900 Avg. Training loss: 5.3468 0.3930 sec/batch
Epoch 1/50 Iteration: 1000 Avg. Training loss: 5.2790 0.3690 sec/batch
Nearest to was: separatist, interwiki, any, previous, angelina, authoritarian, threat, redirects,
Nearest to them: with, crab, thanks, needs, chek, nixon, replaces, kwanzaa,
Nearest to on: interest, asking, biographies, there, hits, cellpadding0styleborder1px, image, plussize,
Nearest to not: stati

Nearest to french: regardsthis, revoking, tends, legitimate, wikipediawide, news, italic, bumper,
Nearest to event: irreversible, sax, banu, oil, sensationalist, mafia, russian, bugle,
Epoch 2/50 Iteration: 4100 Avg. Training loss: 4.5987 0.4149 sec/batch
Epoch 2/50 Iteration: 4200 Avg. Training loss: 4.5783 0.3415 sec/batch
Epoch 3/50 Iteration: 4300 Avg. Training loss: 4.5726 0.1038 sec/batch
Epoch 3/50 Iteration: 4400 Avg. Training loss: 4.5126 0.3497 sec/batch
Epoch 3/50 Iteration: 4500 Avg. Training loss: 4.4956 0.3290 sec/batch
Epoch 3/50 Iteration: 4600 Avg. Training loss: 4.5323 0.3260 sec/batch
Epoch 3/50 Iteration: 4700 Avg. Training loss: 4.5024 0.3452 sec/batch
Epoch 3/50 Iteration: 4800 Avg. Training loss: 4.4706 0.3084 sec/batch
Epoch 3/50 Iteration: 4900 Avg. Training loss: 4.5011 0.3064 sec/batch
Epoch 3/50 Iteration: 5000 Avg. Training loss: 4.4848 0.3576 sec/batch
Nearest to was: interwiki, separatist, angelina, 0341, reacting, authoritarian, 123222396, anyone,
Neares

Epoch 4/50 Iteration: 8100 Avg. Training loss: 4.3629 0.4409 sec/batch
Epoch 4/50 Iteration: 8200 Avg. Training loss: 4.4166 0.4720 sec/batch
Epoch 4/50 Iteration: 8300 Avg. Training loss: 4.3697 0.4894 sec/batch
Epoch 4/50 Iteration: 8400 Avg. Training loss: 4.3987 0.5439 sec/batch
Epoch 4/50 Iteration: 8500 Avg. Training loss: 4.3627 0.4194 sec/batch
Epoch 5/50 Iteration: 8600 Avg. Training loss: 4.1120 0.1973 sec/batch
Epoch 5/50 Iteration: 8700 Avg. Training loss: 4.2411 0.4346 sec/batch
Epoch 5/50 Iteration: 8800 Avg. Training loss: 4.3159 0.4226 sec/batch
Epoch 5/50 Iteration: 8900 Avg. Training loss: 4.3436 0.4117 sec/batch
Epoch 5/50 Iteration: 9000 Avg. Training loss: 4.3210 0.5499 sec/batch
Nearest to was: interwiki, angelina, reacting, 0341, separatist, 123222396, times, finale,
Nearest to them: concerning, kwanzaa, flew, templatenameplease, abrams, 700, 1513, untagged,
Nearest to on: cellpadding0styleborder1px, biographies, futhermore, graciously, four, interest, there, cli

Epoch 6/50 Iteration: 12100 Avg. Training loss: 4.2972 0.2972 sec/batch
Epoch 6/50 Iteration: 12200 Avg. Training loss: 4.2788 0.2914 sec/batch
Epoch 6/50 Iteration: 12300 Avg. Training loss: 4.3111 0.2808 sec/batch
Epoch 6/50 Iteration: 12400 Avg. Training loss: 4.2884 0.2656 sec/batch
Epoch 6/50 Iteration: 12500 Avg. Training loss: 4.2770 0.2662 sec/batch
Epoch 6/50 Iteration: 12600 Avg. Training loss: 4.2668 0.2686 sec/batch
Epoch 6/50 Iteration: 12700 Avg. Training loss: 4.2567 0.2693 sec/batch
Epoch 6/50 Iteration: 12800 Avg. Training loss: 4.2458 0.2716 sec/batch
Epoch 7/50 Iteration: 12900 Avg. Training loss: 3.8080 0.2179 sec/batch
Epoch 7/50 Iteration: 13000 Avg. Training loss: 4.1126 5.6237 sec/batch
Nearest to was: angelina, 0341, reacting, times, interwiki, because, 123222396, collectible,
Nearest to them: concerning, untagged, templatenameplease, kwanzaa, it, sina, flew, orgy,
Nearest to on: biographies, arcade, futhermore, tutoriali, cellpadding0styleborder1px, there, int

Epoch 8/50 Iteration: 16100 Avg. Training loss: 4.1983 0.2985 sec/batch
Epoch 8/50 Iteration: 16200 Avg. Training loss: 4.2085 0.3030 sec/batch
Epoch 8/50 Iteration: 16300 Avg. Training loss: 4.1968 0.3197 sec/batch
Epoch 8/50 Iteration: 16400 Avg. Training loss: 4.2077 0.3277 sec/batch
Epoch 8/50 Iteration: 16500 Avg. Training loss: 4.2187 0.3025 sec/batch
Epoch 8/50 Iteration: 16600 Avg. Training loss: 4.1870 0.2919 sec/batch
Epoch 8/50 Iteration: 16700 Avg. Training loss: 4.1878 0.2763 sec/batch
Epoch 8/50 Iteration: 16800 Avg. Training loss: 4.2190 9.2410 sec/batch
Epoch 8/50 Iteration: 16900 Avg. Training loss: 4.2105 0.3268 sec/batch
Epoch 8/50 Iteration: 17000 Avg. Training loss: 4.2122 8.6561 sec/batch
Nearest to was: angelina, interwiki, times, collectible, 0341, capitalize, reacting, wikipediamultilicensingmultilicense,
Nearest to them: untagged, concerning, templatenameplease, they, thatthere, commonsthe, sina, kwanzaa,
Nearest to on: four, check, interest, biographies, page

...

Epoch 17/50 Iteration: 36100 Avg. Training loss: 4.0447 0.3394 sec/batch
Epoch 17/50 Iteration: 36200 Avg. Training loss: 4.0676 0.3614 sec/batch
Epoch 18/50 Iteration: 36300 Avg. Training loss: 3.9688 0.0152 sec/batch
Epoch 18/50 Iteration: 36400 Avg. Training loss: 3.5106 0.3680 sec/batch
Epoch 18/50 Iteration: 36500 Avg. Training loss: 3.9039 0.2954 sec/batch
Epoch 18/50 Iteration: 36600 Avg. Training loss: 4.0772 0.3014 sec/batch
Epoch 18/50 Iteration: 36700 Avg. Training loss: 4.0384 0.3037 sec/batch
Epoch 18/50 Iteration: 36800 Avg. Training loss: 4.0140 0.2943 sec/batch
Epoch 18/50 Iteration: 36900 Avg. Training loss: 4.0377 0.3141 sec/batch
Epoch 18/50 Iteration: 37000 Avg. Training loss: 4.0417 0.3818 sec/batch
Nearest to was: because, once, hoe, collectible, angelina, ministers, interwiki, vera,
Nearest to them: untagged, they, read, hours, wherever, paddybriggs, reference, contr,
Nearest to on: for, this, talk, four, of, check, and, my,
Nearest to not: that, bs, try, well, s

Epoch 19/50 Iteration: 40100 Avg. Training loss: 3.9972 0.2978 sec/batch
Epoch 19/50 Iteration: 40200 Avg. Training loss: 4.0085 0.3108 sec/batch
Epoch 19/50 Iteration: 40300 Avg. Training loss: 4.0379 0.3565 sec/batch
Epoch 19/50 Iteration: 40400 Avg. Training loss: 4.0256 0.3096 sec/batch
Epoch 19/50 Iteration: 40500 Avg. Training loss: 4.0233 0.3202 sec/batch
Epoch 20/50 Iteration: 40600 Avg. Training loss: 3.6437 0.0963 sec/batch
Epoch 20/50 Iteration: 40700 Avg. Training loss: 3.7632 0.2953 sec/batch
Epoch 20/50 Iteration: 40800 Avg. Training loss: 3.9383 0.3123 sec/batch
Epoch 20/50 Iteration: 40900 Avg. Training loss: 3.9955 0.2954 sec/batch
Epoch 20/50 Iteration: 41000 Avg. Training loss: 4.0242 0.2988 sec/batch
Nearest to was: because, once, 6912122197, hoe, enjoying, cyde, angelina, collectible,
Nearest to them: they, untagged, read, following, hours, useif, contr, have,
Nearest to on: for, this, talk, check, four, page, of, my,
Nearest to not: people, that, it, bs, well, tes

Epoch 21/50 Iteration: 44100 Avg. Training loss: 4.0035 0.4558 sec/batch
Epoch 21/50 Iteration: 44200 Avg. Training loss: 4.0290 0.4189 sec/batch
Epoch 21/50 Iteration: 44300 Avg. Training loss: 4.0267 0.5020 sec/batch
Epoch 21/50 Iteration: 44400 Avg. Training loss: 3.9705 0.4971 sec/batch
Epoch 21/50 Iteration: 44500 Avg. Training loss: 4.0314 0.5131 sec/batch
Epoch 21/50 Iteration: 44600 Avg. Training loss: 4.0039 0.4899 sec/batch
Epoch 21/50 Iteration: 44700 Avg. Training loss: 4.0204 0.5599 sec/batch
Epoch 21/50 Iteration: 44800 Avg. Training loss: 4.0037 0.7317 sec/batch
Epoch 22/50 Iteration: 44900 Avg. Training loss: 3.4949 0.2973 sec/batch
Epoch 22/50 Iteration: 45000 Avg. Training loss: 3.8260 0.4833 sec/batch
Nearest to was: because, once, hoe, just, were, got, angelina, shop,
Nearest to them: they, untagged, read, have, hours, that, after, these,
Nearest to on: for, this, talk, and, my, of, a, to,
Nearest to not: that, to, people, it, but, from, try, is,
Nearest to use: fai

Epoch 23/50 Iteration: 48100 Avg. Training loss: 4.0015 0.3592 sec/batch
Epoch 23/50 Iteration: 48200 Avg. Training loss: 3.9742 0.3548 sec/batch
Epoch 23/50 Iteration: 48300 Avg. Training loss: 3.9860 0.3867 sec/batch
Epoch 23/50 Iteration: 48400 Avg. Training loss: 3.9863 0.3958 sec/batch
Epoch 23/50 Iteration: 48500 Avg. Training loss: 4.0282 0.3837 sec/batch
Epoch 23/50 Iteration: 48600 Avg. Training loss: 4.0160 0.4274 sec/batch
Epoch 23/50 Iteration: 48700 Avg. Training loss: 3.9751 0.3789 sec/batch
Epoch 23/50 Iteration: 48800 Avg. Training loss: 3.9923 0.3688 sec/batch
Epoch 23/50 Iteration: 48900 Avg. Training loss: 3.9703 0.4266 sec/batch
Epoch 23/50 Iteration: 49000 Avg. Training loss: 3.9998 0.4417 sec/batch
Nearest to was: because, once, angelina, hoe, were, raja, collectible, 2014the,
Nearest to them: they, untagged, after, satirical, useif, read, have, contr,
Nearest to on: for, talk, this, and, check, page, my, you,
Nearest to not: to, that, well, from, is, want, with, 

Epoch 25/50 Iteration: 52200 Avg. Training loss: 3.9991 0.4388 sec/batch


In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('../data/checkpoints'))
    embed_mat = sess.run(embedding)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
ax.axis([-200.0,200.0, -200.0,200.0])
ax = fig.gca()
ax.set_autoscale_on(False)

for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)
plt.savefig('w2v_3_AllWords.png')